"Electric Reliability Council of Texas (ERCOT) manages the flow of electric power to more than 25 million Texas customers -- representing about 90 percent of the state’s electric load." (<a href="http://www.ercot.com/about">source</a>)

Example report: http://www.ercot.com/content/wcm/lists/143010/2018_Long-Term_Hourly_Peak_Demand_and_Energy_Forecast_Final.pdf

Data source:<BR>
http://www.ercot.com/gridinfo/load/load_hist/

In [1]:
!pip install beautifulsoup4

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import pandas
print('pandas',pandas.__version__)
import glob
import zipfile
from statsmodels.tsa.seasonal import seasonal_decompose
import random
import time
import matplotlib.pyplot as plt

pandas 0.23.4


In [3]:
domain = "http://www.ercot.com"

In [4]:
r = requests.get(domain+"/gridinfo/load/load_hist/")

In [5]:
soup = BeautifulSoup(r.content)

In [6]:
# absolute links
for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
    print(link.get('href'))

http://lists.ercot.com
http://www.twitter.com/ercot_iso
http://www.linkedin.com/company/ercot
http://www.youtube.com/user/ERCOTISO?feature=mhee


In [7]:
# absolute links
for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
    print(link.get('href'))

https://www2.virtualtrainingassistant.com/Ercot/LearnerConnection/Security/Logon.aspx
https://mis.ercot.com/marketrak/tmtrack/tmtrack.dll?
https://mis.ercot.com/pps/tibco/mis
https://www.texasrenewables.com
https://etod.ercot.com
https://www.facebook.com/pages/Electric-Reliability-Council-of-Texas/208282585857723


In [8]:
# absolute links
#for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
# relative links
for link in soup.findAll('a'):
    print(link.get('href'))

/
/news/releases
/careers
/services/comm/feeds
/about/contact
/about
/services
/committees
/mktrules
/mktinfo
/gridinfo
/
/gridinfo
/gridinfo/load
/gridinfo/csc
/gridinfo/generation
/gridinfo/load
/gridinfo/load/load_hist
/gridinfo/load/forecast
/gridinfo/transmission
/gridinfo/resource
/gridinfo/sysplan
/gridinfo/planning
/content/wcm/key_documents_lists/89022/Native_Load_2019.zip
/content/wcm/key_documents_lists/89022/native_load_2018.zip
/content/wcm/key_documents_lists/89022/native_load_2017.zip
/content/wcm/key_documents_lists/89022/native_Load_2016.zip
/content/gridinfo/load/load_hist/native_Load_2015.xls
/content/gridinfo/load/load_hist/2014_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2013_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2012_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2011_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2010_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2009_ERCOT_Hourly_Load_

now that I've found the files as relative paths, I want to filter the above results to include only .zip and .xls files

In [9]:
for link in soup.findAll('a'):
    link_path = link.get('href')
    if 'xls' in link_path:
        print(link_path)
        xls_link = link_path
    if '.zip' in link_path:
        print(link_path)

/content/wcm/key_documents_lists/89022/Native_Load_2019.zip
/content/wcm/key_documents_lists/89022/native_load_2018.zip
/content/wcm/key_documents_lists/89022/native_load_2017.zip
/content/wcm/key_documents_lists/89022/native_Load_2016.zip
/content/gridinfo/load/load_hist/native_Load_2015.xls
/content/gridinfo/load/load_hist/2014_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2013_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2012_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2011_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2010_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2009_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2008_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2007_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2006_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2005_ERCOT_Hourly_Load_Data.xls
/content/gridinfo/load/load_hist/2004_ERCOT_Hourly_Load_

as a test before creating more code, let's download one of those files

In [10]:
url = domain+xls_link
print(url)
r = requests.get(url, allow_redirects=True)
open('filename.xls', 'wb').write(r.content)

http://www.ercot.com/content/gridinfo/load/load_hist/erceei97.xls


109056

just to verify the contents, take a look at the dataframe

In [11]:
df = pandas.read_excel('filename.xls')
print(df.shape)
df.head()

(368, 25)


,ERCOT - Composite Load,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,--- Hour Ending ---,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00,00:00:00
3,1997-01-01 00:00:00,19662,18931,18162,17772,17636,17876,18328,18541,19078,...,21382,21294,21331,22261,23890,23613,23080,22373,21170,19576
4,1997-01-02 00:00:00,18414,17757,17393,17343,17648,18929,21082,22731,23383,...,25144,25147,24963,25309,26693,26259,25561,24462,22898,21102


Now that we are confident based on one sample, get all the data

In [12]:
def download_file(url,filename):
    """
    download a file based on a URL, save to filename
    """
    if not isinstance(url,str):
        raise TypeError("url must be string")
    if not isinstance(filename,str):
        raise TypeError("filename must be string")
        
    r = requests.get(url, allow_redirects=True)
    open(filename, 'wb').write(r.content)
    return

In [13]:
for link in soup.findAll('a'):
    link_path = link.get('href')
    filename = link_path.split('/')[-1]
    if ('.xls' in link_path) or ('.zip' in link_path):
        print(filename)
        download_file(domain+link_path,filename)

Native_Load_2019.zip
native_load_2018.zip
native_load_2017.zip
native_Load_2016.zip
native_Load_2015.xls
2014_ERCOT_Hourly_Load_Data.xls
2013_ERCOT_Hourly_Load_Data.xls
2012_ERCOT_Hourly_Load_Data.xls
2011_ERCOT_Hourly_Load_Data.xls
2010_ERCOT_Hourly_Load_Data.xls
2009_ERCOT_Hourly_Load_Data.xls
2008_ERCOT_Hourly_Load_Data.xls
2007_ERCOT_Hourly_Load_Data.xls
2006_ERCOT_Hourly_Load_Data.xls
2005_ERCOT_Hourly_Load_Data.xls
2004_ERCOT_Hourly_Load_Data.xls
2003_ERCOT_Hourly_Load_Data.xls
2002_ERCOT_Hourly_Load_Data.xls
2000FERC714.zip
1999FERC714.zip
FERC714.zip
97frc714.zip
erceei97.xls


what is now in my local directory?

In [14]:
!ls

 1999FERC714.zip		   2013_ERCOT_Hourly_Load_Data.xls
 2000FERC714.zip		   2014_ERCOT_Hourly_Load_Data.xls
 2002_ERCOT_Hourly_Load_Data.xls   97frc714.zip
 2003_ERCOT_Hourly_Load_Data.xls   erceei97.xls
 2004_ERCOT_Hourly_Load_Data.xls   FERC714.zip
 2005_ERCOT_Hourly_Load_Data.xls   filename.xls
 2006_ERCOT_Hourly_Load_Data.xls  'historical power load.ipynb'
 2007_ERCOT_Hourly_Load_Data.xls   native_Load_2015.xls
 2008_ERCOT_Hourly_Load_Data.xls   native_Load_2016.zip
 2009_ERCOT_Hourly_Load_Data.xls   native_load_2017.zip
 2010_ERCOT_Hourly_Load_Data.xls   native_load_2018.zip
 2011_ERCOT_Hourly_Load_Data.xls   Native_Load_2019.zip
 2012_ERCOT_Hourly_Load_Data.xls


we'll need to extract the .zip files

In [15]:
list_of_zips = glob.glob('*.zip')
list_of_zips

['native_Load_2016.zip',
 'native_load_2017.zip',
 'Native_Load_2019.zip',
 'native_load_2018.zip',
 '2000FERC714.zip',
 '1999FERC714.zip',
 'FERC714.zip',
 '97frc714.zip']

In [16]:
for zip_file in list_of_zips:
#    print(zip_file)
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        zip_ref.extractall(zip_file+"_folder")

In [24]:
!ls -hal

total 30M
drwxr-xr-x 36 jovyan users 1.2K Mar 23 16:52  .
drwxr-xr-x 12 jovyan users  384 Mar 23 16:30  ..
-rw-r--r--  1 jovyan users 373K Mar 23 16:52  1999FERC714.zip
drwxr-xr-x  6 jovyan users  192 Mar 23 16:52  1999FERC714.zip_folder
-rw-r--r--  1 jovyan users 373K Mar 23 16:52  2000FERC714.zip
drwxr-xr-x  6 jovyan users  192 Mar 23 16:52  2000FERC714.zip_folder
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2002_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2003_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2004_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2005_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2006_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2007_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2008_ERCOT_Hourly_Load_Data.xls
-rw-r--r--  1 jovyan users 1.8M Mar 23 16:52  2009_ERCOT_Hourly_Load_Data.xls
-rw-r--

In [3]:
for file_name in glob.glob('20*.xls'):
    df = pandas.read_excel(file_name)
    print('\n'+file_name)
    print(df.columns) # verified that columns are consistent
    print(df.shape)


2006_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT'],
      dtype='object')
(8760, 10)

2010_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT'],
      dtype='object')
(8760, 10)

2007_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT'],
      dtype='object')
(8760, 10)

2011_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT'],
      dtype='object')
(8760, 10)

2013_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 'NORTH_C', 'SOUTHERN',
       'SOUTH_C', 'WEST', 'ERCOT'],
      dtype='object')
(8760, 10)

2005_ERCOT_Hourly_Load_Data.xls
Index(['Hour_End', 'COAST', 'EAST', 'FAR_WEST', 'NORTH', 

In [4]:
start_time=time.time()
list_of_df = []
for file_name in glob.glob('20*.xls'):
    df = pandas.read_excel(file_name)
    list_of_df.append(df)
    
df = pandas.concat(list_of_df)
df.shape
print('elapsed:',time.time()-start_time,'seconds')

elapsed: 9.0895414352417 seconds


In [5]:
df.head()

,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT
0,2006-01-01 01:00:00.003,7623.394894,788.305875,1026.559049,777.157535,7549.809438,1852.416802,3797.223989,639.553370,24054.420954
1,2006-01-01 01:59:59.997,7437.102525,779.587442,1024.320168,752.171350,7291.494704,1770.283786,3637.505535,628.083900,23320.549410
2,2006-01-01 03:00:00.000,7151.481479,767.310985,1011.297406,725.028737,7012.269706,1676.891230,3482.207530,614.014931,22440.502004
3,2006-01-01 04:00:00.003,6957.302786,774.671447,1004.026250,711.968331,6896.466664,1609.080496,3363.417807,611.227308,21928.161089
4,2006-01-01 04:59:59.997,6861.596790,772.916673,1005.141879,708.933070,6810.395549,1595.417621,3303.547499,613.746434,21671.695516


In [6]:
df.to_pickle('power_data.pkl')